<a href="https://colab.research.google.com/github/ele9996/keyword-spotting/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

In [29]:
import pathlib
pathlib.Path().absolute()

PosixPath('/content')

In [30]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#parser.add_argument('--silence', action='store_true', help='add silence')
#args = parser.parse_args()

mfcc=True
silence=False

In [31]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#Data extraction from file

In [32]:
#if args.silence is True:
if silence is True:
    data_dir = os.path.join('.', 'data', 'mini_speech_commands_silence')
else:
    zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')

    data_dir = os.path.join('.', 'data', 'mini_speech_commands')

filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

#if args.silence is True:
if silence is True:
    total = 9000
else:
    total = 8000

#Training, Validation and Test dataset definition

In [33]:
train_files = filenames[:int(total*0.8)]
val_files = filenames[int(total*0.8): int(total*0.9)]
test_files = filenames[int(total*0.9):]

In [34]:
LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

#Data preparation and preprocessing steps

In [35]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [36]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [37]:
#if args.mfcc is True:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]


In [38]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

#if args.silence is True:
if silence is True:
    units = 9
else:
    units = 8

#Model application

###MPL

In [39]:
if (mfcc==True):
  path= './callback_val_chkp/mfcc_mpl'
else:
  path= './callback_val_chkp/stft_mpl'
  
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)



In [40]:
#MLP Model definition
mlp_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=8) 
])

In [41]:
#Training, evaluation and testing with mlp model


mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
mlp_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])



Epoch 1/20
200/200 [==============================] - 94s 471ms/step - loss: 1.7451 - accuracy: 0.4319 - val_loss: 1.2901 - val_accuracy: 0.5587
Epoch 2/20
200/200 [==============================] - 1s 7ms/step - loss: 0.9681 - accuracy: 0.6517 - val_loss: 1.0147 - val_accuracy: 0.6612
Epoch 3/20
200/200 [==============================] - 1s 7ms/step - loss: 0.7267 - accuracy: 0.7428 - val_loss: 0.9122 - val_accuracy: 0.7225
Epoch 4/20
200/200 [==============================] - 2s 8ms/step - loss: 0.5882 - accuracy: 0.7870 - val_loss: 0.7859 - val_accuracy: 0.7462
Epoch 5/20
200/200 [==============================] - 1s 7ms/step - loss: 0.4868 - accuracy: 0.8252 - val_loss: 0.7353 - val_accuracy: 0.7550
Epoch 6/20
200/200 [==============================] - 1s 4ms/step - loss: 0.4137 - accuracy: 0.8520 - val_loss: 0.8291 - val_accuracy: 0.7400
Epoch 7/20
200/200 [==============================] - 1s 4ms/step - loss: 0.3335 - accuracy: 0.8761 - val_loss: 0.7515 - val_accuracy: 0.7738
Epo

In [42]:
#model evaluation on validation set
mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= mlp_model.evaluate(val_ds)

25/25 [==============================] - 0s 1ms/step - loss: 1.7038


In [43]:
test_loss= mlp_model.evaluate(test_ds)

25/25 [==============================] - 18s 714ms/step - loss: 1.5832


###CNN

In [44]:
if (mfcc==True):
  path= './callback_val_chkp/mfcc_cnn'
else:
  path= './callback_val_chkp/stft_cnn'

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)


In [45]:
#CNN_2D Model definition

cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [46]:
#Training, evaluation and testing with mlp model
cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
cnn_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
200/200 [==============================] - 36s 178ms/step - loss: 1.2493 - accuracy: 0.6125 - val_loss: 0.7830 - val_accuracy: 0.7800
Epoch 2/20
200/200 [==============================] - 36s 181ms/step - loss: 0.6274 - accuracy: 0.8375 - val_loss: 0.5097 - val_accuracy: 0.8525
Epoch 3/20
200/200 [==============================] - 36s 181ms/step - loss: 0.4379 - accuracy: 0.8873 - val_loss: 0.3911 - val_accuracy: 0.8813
Epoch 4/20
200/200 [==============================] - 35s 177ms/step - loss: 0.3483 - accuracy: 0.9053 - val_loss: 0.3550 - val_accuracy: 0.8913
Epoch 5/20
200/200 [==============================] - 36s 179ms/step - loss: 0.2824 - accuracy: 0.9250 - val_loss: 0.2807 - val_accuracy: 0.9125
Epoch 6/20
200/200 [==============================] - 34s 168ms/step - loss: 0.2414 - accuracy: 0.9348 - val_loss: 0.3346 - val_accuracy: 0.8938
Epoch 7/20
200/200 [==============================] - 35s 175ms/step - loss: 0.2194 - accuracy: 0.9409 - val_loss: 0.2720 - val_ac

In [47]:
#model evaluation on validation set
cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= cnn_model.evaluate(val_ds)

25/25 [==============================] - 1s 37ms/step - loss: 1.1708


In [48]:
#model test on the test set
test_loss= cnn_model.evaluate(test_ds)

25/25 [==============================] - 1s 37ms/step - loss: 1.2796


###DS-CNN

In [49]:
if (mfcc==True):
  path= './callback_val_chkp/mfcc_DS-CNN'
else:
  path= './callback_val_chkp/stft_DS-CNN'

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=path,
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)


In [50]:
ds_cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=256,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [51]:
#Training, evaluation and testing with mlp model
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
ds_cnn_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
200/200 [==============================] - 29s 144ms/step - loss: 1.2148 - accuracy: 0.6172 - val_loss: 0.6894 - val_accuracy: 0.8075
Epoch 2/20
200/200 [==============================] - 29s 143ms/step - loss: 0.5894 - accuracy: 0.8384 - val_loss: 0.5196 - val_accuracy: 0.8450
Epoch 3/20
200/200 [==============================] - 28s 141ms/step - loss: 0.3947 - accuracy: 0.8913 - val_loss: 0.3887 - val_accuracy: 0.8850
Epoch 4/20
200/200 [==============================] - 29s 143ms/step - loss: 0.3245 - accuracy: 0.9069 - val_loss: 0.3746 - val_accuracy: 0.8712
Epoch 5/20
200/200 [==============================] - 29s 143ms/step - loss: 0.2782 - accuracy: 0.9181 - val_loss: 0.2997 - val_accuracy: 0.9175
Epoch 6/20
200/200 [==============================] - 29s 143ms/step - loss: 0.2318 - accuracy: 0.9331 - val_loss: 0.2922 - val_accuracy: 0.9162
Epoch 7/20
200/200 [==============================] - 29s 145ms/step - loss: 0.2120 - accuracy: 0.9367 - val_loss: 0.2542 - val_ac

In [52]:
#model evaluation on validation set
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= ds_cnn_model.evaluate(val_ds)

25/25 [==============================] - 1s 32ms/step - loss: 0.6302


In [53]:
#model test on the test set
test_loss= ds_cnn_model.evaluate(test_ds)

25/25 [==============================] - 1s 32ms/step - loss: 0.8149


In [60]:
!zip -r model.zip {"./callback_val_chkp"}

  adding: callback_val_chkp/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/assets/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/saved_model.pb (deflated 89%)
  adding: callback_val_chkp/stft_mpl/variables/ (stored 0%)
  adding: callback_val_chkp/stft_mpl/variables/variables.index (deflated 66%)
  adding: callback_val_chkp/stft_mpl/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: callback_val_chkp/stft_cnn/ (stored 0%)
  adding: callback_val_chkp/stft_cnn/assets/ (stored 0%)
  adding: callback_val_chkp/stft_cnn/saved_model.pb (deflated 91%)
  adding: callback_val_chkp/stft_cnn/variables/ (stored 0%)
  adding: callback_val_chkp/stft_cnn/variables/variables.index (deflated 69%)
  adding: callback_val_chkp/stft_cnn/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: callback_val_chkp/mfcc_mpl/ (stored 0%)
  adding: callback_val_chkp/mfcc_mpl/assets/ (stored 0%)
  adding: callback_val_chkp/mfcc_mpl/saved_mod

In [61]:
!ls

callback_val_chkp  data  model.zip  sample_data


In [63]:
try:
  from google.colab import files
  files.download('./model.zip')
except ImportError:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>